# LSI Word Embedding using K-means, Gaussian Mixture and Cosine Similarity as Clustering

In [3]:
import os.path

from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# from clustering_class import *
# from new_combine_models import *
# from DEC import *
# from generate_word_vector import *

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [65]:
%%html
<style>
table {float:center}
</style>

In [18]:
def preprocess_data(doc_list):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = set(stopwords.words('english'))# create English stop words list
    p_stemmer = PorterStemmer()
    processed_doc_list = []

    for doc in doc_list:
        lower_doc = doc.lower()
        token_list = tokenizer.tokenize(lower_doc)
        stopped_token_list = [i for i in token_list if not i in en_stop]
        stemmed_token_list = [p_stemmer.stem(i) for i in stopped_token_list]
        processed_doc = ' '.join(stemmed_token_list) # Untokenize
        processed_doc_list.append(processed_doc)
    
    return processed_doc_list

In [38]:
df = pd.read_csv('yahoo_spiders/data_out/yahoo_desc.csv', ).dropna().reset_index()

In [39]:
df.head()

,index,Ticker,Description,Sector,Industry
0,0,ABT,"Abbott Laboratories discovers, develops, manuf...",Healthcare,Medical Devices
1,1,ACN,"Accenture plc provides consulting, technology,...",Technology,Information Technology Services
2,2,MMM,"3M Company develops, manufactures, and markets...",Industrials,Specialty Industrial Machinery
3,3,ABMD,"Abiomed, Inc. engages in the research, develop...",Healthcare,Medical Devices
4,4,ADBE,Adobe Inc. operates as a diversified software ...,Technology,Software—Infrastructure


In [40]:
df.shape

(438, 5)

In [41]:
len(set(df.Sector))

11

In [42]:
len(set(df.Industry))

109

In [43]:
df.iloc[1].Description

"Accenture plc provides consulting, technology, and outsourcing services worldwide. Its Communications, Media & Technology segment provides professional services for clients to accelerate and deliver digital transformation, develop industry-specific solutions, and enhance efficiencies and business results for communications, media, high tech, software, and platform companies. The company's Financial Services segment offers services for profitability pressures, industry consolidation, regulatory changes, and the need to continually adapt to new digital technologies for banking, capital market, and insurance industries. Its Health & Public Service segment provides consulting services and digital solutions to help clients deliver social, economic, and health outcomes for healthcare payers and providers, government departments and agencies, public service organizations, educational institutions, and non-profit organizations. The company's Products segment helps clients enhance their perfor

In [44]:
desc_list = df['Description']

##### Pre Processing

In [46]:
preped_desc_list = preprocess_data(desc_list)

In [48]:
len(preped_desc_list)

438

##### Tdidf vectorization

In [49]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= None, max_df = 0.5, smooth_idf=True)
dtm = vectorizer.fit_transform(preped_desc_list) # dtm - Document Matrix (sparse matrix)
dtm.shape

(438, 7263)


| Index | Unique Words (features)
| :- | -: | 
| Russell | 20025 |
| SnP | 7263 |

##### LSA Word Embedding

In [55]:
# Test (Actual For loop is below)
lsa = TruncatedSVD(200, algorithm = 'arpack')
dtm_lsa = lsa.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa) # normalisae so each vector is len 1

In [56]:
pd.DataFrame(dtm_lsa).head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.186027,-0.063461,-0.212476,0.132797,0.086120,0.222043,0.301794,0.259920,-0.031192,0.080088,...,-0.067338,0.004449,-0.056364,-0.038719,-0.017974,-0.009489,0.007618,0.073074,-0.008122,0.052722
1,0.401177,-0.015788,-0.164992,-0.057852,0.110736,-0.037090,-0.000830,0.066506,-0.030720,-0.064347,...,0.126713,-0.066480,-0.001598,-0.062942,-0.011841,-0.032356,0.002961,0.006440,0.045562,-0.038511
2,0.324800,-0.131666,-0.206526,0.105576,-0.036015,0.238173,0.060495,0.028622,-0.039046,-0.012727,...,-0.005368,0.030757,-0.042402,0.006760,-0.032632,0.000339,-0.027574,-0.002905,-0.015595,-0.019584
3,0.110320,-0.053520,-0.118458,-0.013129,0.035338,0.047525,0.083328,0.038494,0.032770,0.062653,...,-0.049930,-0.051575,0.153006,-0.111204,0.006802,-0.052839,0.030731,0.022081,0.072314,0.070727
4,0.267790,-0.002623,-0.201545,-0.078364,-0.025053,-0.240672,-0.079426,0.104835,-0.052271,-0.000145,...,-0.008892,-0.023900,-0.023031,0.017898,-0.017054,0.032710,0.074666,0.017683,0.000755,-0.039082


# Clustering Algos

In [57]:
NUM_INDUSTRIES = 69

### K - Means

In [58]:
# defining the kmeans function with initialization as k-means++
kmeans = KMeans(n_clusters=NUM_INDUSTRIES, init='k-means++')
# fitting the k means algorithm on scaled data
kmeans.fit(dtm_lsa)
pred = kmeans.predict(dtm_lsa)
# pd.concat([df[['company']], pd.DataFrame(pred)], axis=1).head()
pred

array([10, 53,  8, 19, 43, 43, 50, 43, 52, 10,  2, 17,  3, 60, 42, 40, 48,
       39, 27, 20, 17, 56,  3, 59, 43, 40, 17, 41, 56, 39, 64, 24, 28, 38,
       30, 50,  2, 28,  2, 64, 26, 25, 23, 14, 67, 37,  3, 56, 56, 62, 13,
       28, 18, 52, 23, 33,  8, 51, 63,  6, 50,  5, 25, 12, 44, 34, 39, 19,
       16, 21, 38, 68, 26,  9, 30, 63, 24, 14, 54, 36, 34, 31, 61, 62, 38,
        5, 60,  6, 13, 11, 46, 65,  2, 63, 56, 63,  8, 28, 31, 46, 62, 64,
       53,  9,  9, 26, 13, 62, 26, 59, 24, 14, 25,  0, 67, 38, 37, 34, 55,
       20, 65,  5, 10, 29, 26, 27, 40, 26, 16, 13, 13, 41, 25, 13, 62, 57,
        1, 62, 17, 61, 35, 67, 53, 34,  6, 31, 64, 61, 62, 19, 43, 28, 17,
       26, 21, 23, 23,  5, 17, 12, 56, 17, 15,  3, 11, 29, 35, 43, 68, 21,
       63, 63,  4, 41,  0,  1, 41, 14,  3,  4, 20, 13, 13, 31, 22,  9, 16,
       53, 14, 44,  2, 29, 41,  6, 29, 20, 33, 32, 43,  5, 56, 39, 27, 26,
       47, 28, 42, 10, 58,  9, 39, 44, 67,  2, 63, 22,  1, 29, 10, 48, 30,
        1, 53, 31, 51, 37

In [59]:
def kmeans_cluster(dtm):
    '''
    input: Document Matrix
    output: list indicating clusters
    '''
    kmeans = KMeans(n_clusters=NUM_INDUSTRIES, init='k-means++')
    kmeans.fit(dtm)
    pred = kmeans.predict(dtm)
    return pred

### Gaussian Mixture

In [60]:
def gmm_cluster(dtm):
    '''
    input: Document Matrix
    output: list indicating clusters
    '''
    gmm = GaussianMixture(n_components=NUM_INDUSTRIES)
    gmm.fit(dtm)
    pred = gmm.predict(dtm)
    return pred

### Cosine Similarity

In [ ]:
x = [((1,),(2,),3), ((4,),(1458,),6)]

mcorr = ((1458,), (1830,), 1.0000000000000004)
list(filter(lambda tup: (tup[0] not in mcorr) and (tup[1] not in mcorr), x))

In [ ]:
((1,2), (4,)) in (1,)

In [61]:
def cosine_similarity(dtm):
    '''
    input: Doc Matrix
    output: list of lists indicating clusters
    '''
    corr = np.asarray(np.asmatrix(dtm) * np.asmatrix(dtm).T)
    L = corr.shape[0]
    
    cluster_corrs = []
    for i in range(L):
        for j in range(i+1,L):
            corr_tuple = ((i,), (j,), corr[i][j])
            cluster_corrs.append(corr_tuple)
            
    cluster_list = [(i,) for i in range(L)]
    
    num_iter = L - NUM_INDUSTRIES
    for x in range(num_iter):
        max_corr_tup = max(cluster_corrs, key = lambda tup: tup[2])
        # filters ur max_corr_tup also
        print(max_corr_tup)
        cluster_corrs = list(filter(lambda tup: (tup[0] not in max_corr_tup) and (tup[1] not in max_corr_tup),cluster_corrs))
        new_cluster = max_corr_tup[0] + max_corr_tup[1] # concatenate tups
        cluster_list.remove(max_corr_tup[0])
        cluster_list.remove(max_corr_tup[1])
        for cluster in cluster_list:
            # Similarity
            n = len(new_cluster) * len(cluster)
            total_similarity = 0
            for cpy_1 in new_cluster:
                for cpy_2 in cluster:
                    total_similarity += corr[cpy_1][cpy_2]
            similarity = total_similarity / n
            corr_tuple = (new_cluster, cluster, similarity)
            cluster_corrs.append(corr_tuple)
        cluster_list.append(new_cluster)
    
    if len(cluster_list) != NUM_INDUSTRIES:
        raise Exception()
        
    pred = np.full(N, -1)
    for i in range(NUM_INDUSTRIES):
        cluster = list(cluster_list[i])
        pred[cluster] = [i] * len(cluster)
    return pred

In [62]:
cosine_similarity(dtm_lsa)

((299,), (301,), 1.0000000000000007)
((400,), (401,), 1.0000000000000002)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
dtmx = dtm_lsa
corr = np.asarray(np.asmatrix(dtmx) * np.asmatrix(dtmx).T)
L = corr.shape[0]

cluster_corrs = []
for i in range(L):
    for j in range(i+1,L):
        corr_tuple = ((i,), (j,), corr[i][j])
        cluster_corrs.append(corr_tuple)

cluster_list = [(i,) for i in range(L)]

num_iter = L - NUM_INDUSTRIES
for x in range(num_iter):
    max_corr_tup = max(cluster_corrs, key = lambda tup: tup[2])
    # filters ur max_corr_tup also
    print(max_corr_tup)
    cluster_corrs = list(filter(lambda tup: (tup[0] not in max_corr_tup) and (tup[1] not in max_corr_tup),cluster_corrs))
    new_cluster = max_corr_tup[0] + max_corr_tup[1] # concatenate tups
    cluster_list.remove(max_corr_tup[0])
    cluster_list.remove(max_corr_tup[1])
    for cluster in cluster_list:
        # Similarity
        n = len(new_cluster) * len(cluster)
        total_similarity = 0
        for cpy_1 in new_cluster:
            for cpy_2 in cluster:
                total_similarity += corr[cpy_1][cpy_2]
        similarity = total_similarity / n
        corr_tuple = (new_cluster, cluster, similarity)
        cluster_corrs.append(corr_tuple)
    cluster_list.append(new_cluster)

if len(cluster_list) != NUM_INDUSTRIES:
    raise Exception()

pred = np.full(N, -1)
for i in range(NUM_INDUSTRIES):
    cluster = list(cluster_list[i])
    pred[cluster] = [i] * len(cluster)

In [ ]:
def t(tup):
    print((tup[0] not in max_corr_tup), (tup[1] not in max_corr_tup))
    return (tup[0] not in max_corr_tup) and (tup[1] not in max_corr_tup)
list(filter(t,cluster_corrs))

### Run all Clustering algos

In [63]:
cluster_dfs_list = [df[['Ticker']]]
for k in range(200, 300, 100):
    print(k)
    lsa_k = TruncatedSVD(k, algorithm = 'arpack')
    dtm_lsa_k = lsa_k.fit_transform(dtm)
    dtm_lsa_k = Normalizer(copy=False).fit_transform(dtm_lsa_k)
    # K-Means
    clusters_k = kmeans_cluster(dtm_lsa_k)
    cluster_dfs_list.append(pd.DataFrame(clusters_k, columns=['lsi_{}_kmeans'.format(str(k))]))
    # GMM
    clusters_k = gmm_cluster(dtm_lsa_k)
    cluster_dfs_list.append(pd.DataFrame(clusters_k, columns=['lsi_{}_gmm'.format(str(k))]))
    # Cosine_similarity
    
all_cluster = pd.concat(cluster_dfs_list, axis=1)

200


In [64]:
all_cluster.head()

,Ticker,lsi_200_kmeans,lsi_200_gmm
0,ABT,23,34
1,ACN,39,25
2,MMM,1,33
3,ABMD,10,35
4,ADBE,43,19


## Evaluation

##### Preparation

In [23]:
stock_data = pd.read_csv('data/price_us.csv',index_col=0)
stock_data.index.names = ['date']
stock_data.index = pd.to_datetime(stock_data.index,format='%Y%m%d')
stock_data = stock_data[stock_data.columns.intersection(all_cluster['company'].values.tolist())]

In [24]:
stock_data.head()

,A,AA,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,AAWW,...,ZION,ZIOP,ZIXI,ZNGA,ZS,ZTS,ZUMZ,ZUO,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,38.5539,34.8453,51.0799,30.0870,10.79,21.0696,156.793,99.946,35.2148,48.42,...,25.7582,5.13,3.55,2.73,16.00,41.6883,38.21,14.00,14.00,0.175245
2015-01-05,37.8315,32.8266,51.0467,29.9691,10.65,20.3538,154.727,97.130,35.6161,46.65,...,24.7931,5.07,3.48,2.71,33.00,41.4380,38.94,20.00,16.25,0.155652
2015-01-06,37.2422,33.0679,50.2556,28.8489,10.25,20.0319,154.618,97.139,35.8952,45.66,...,23.8462,4.96,3.39,2.70,27.90,41.0338,38.46,20.60,19.32,0.145916
2015-01-07,37.7365,33.9237,50.2272,29.7235,9.85,20.2145,157.940,98.501,36.5756,46.48,...,24.0693,5.02,3.38,2.68,30.38,41.8808,40.28,20.60,24.54,0.142995
2015-01-08,38.8676,34.8892,50.8430,30.2049,9.96,20.7141,159.325,102.286,36.7937,48.21,...,24.4107,4.99,3.64,2.58,31.08,42.5257,41.40,19.55,35.14,0.141048


In [25]:
stock_data = stock_data.pct_change()
stock_data=stock_data.stack()
stock_data = stock_data.reset_index()
stock_data.columns = ['date','company','return']
stock_data = stock_data[['company','return','date']]

In [26]:
stock_data.head()

,company,return,date
0,A,-0.018737,2015-01-05
1,AA,-0.057933,2015-01-05
2,AAL,-0.000650,2015-01-05
3,AAN,-0.003919,2015-01-05
4,AAOI,-0.012975,2015-01-05


##### Results

In [29]:
performance=performance_analysis(class_df=all_cluster,return_df=stock_data)
performance.get_statistical_describe()

C:\Users\sival\Desktop\RMI\industry-classification-master\industry-classification-master\new_combine_models.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  1, thresh=ret_comp, inplace=True)
c:\users\sival\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\sival\Desktop\RMI\industry-classification-master\industry-classification-master\new_combine_models.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [35]:
performance.print_table().sort_index()

summary table:


,R2,proportion of right classification,classes number
lsi_200_gmm,0.434857,0.732819,69.0
lsi_200_kmeans,0.428494,0.723001,69.0
lsi_300_gmm,0.431955,0.729663,69.0
lsi_300_kmeans,0.444555,0.717742,69.0
lsi_400_gmm,0.443627,0.719846,69.0
lsi_400_kmeans,0.438421,0.717391,69.0
lsi_500_gmm,0.433254,0.700561,69.0
lsi_500_kmeans,0.449381,0.731066,69.0
lsi_600_gmm,0.435887,0.708626,69.0
lsi_600_kmeans,0.437951,0.714236,69.0


|Embedding model|R2|proportion of right classification|
|----------------|--------|--------|
| LSI length 200 | 0.4537 | 0.4962 |
| LSI length 300 | 0.4459 | 0.4768 |
| LSI length 400 | 0.4421 | 0.4072 |
| LSI length 500 | 0.4554 | 0.4244 |
| LSI length 600 | 0.4362 | 0.4409 |
| LSI length 700 | 0.4407 | 0.3842 |
| LSI length 800 | 0.4417 | 0.4921 |
| LSI length 900 | 0.4478 | 0.4005 |
| LSI length 1000 | 0.4403 | 0.4068 |
| LSI length 1100 | 0.4380 | 0.3933 |

- not sure why in their code, the classes number is not 69

In [ ]:
performance.plot_industry_dense()

In [31]:
ratio_df = pd.read_csv('data/russell3000_ratios.csv')
def multiplier(num):
    out=np.nan
    if num[-1]=='B':
        out=float(num[:-1])*1000
    elif num[-1]=='T':
        out=float(num[:-1])*1000*1000
    elif num[-1]=='M':
        out=float(num[:-1])
    return out
ratio_df['mkt_cap']=ratio_df['mkt_cap'].map(multiplier)
ratio_df['beta'] = pd.to_numeric(ratio_df['beta'],errors='coerce')
ratio_df['profit_m']=ratio_df['profit_m'].map(lambda x: float(x.strip('%').replace(',',''))/100 if not pd.isnull(x) else np.nan)
ratio_df['roa']=ratio_df['roa'].map(lambda x: float(x.strip('%').replace(',',''))/100 if not pd.isnull(x) else np.nan)
ratio_df['roe']=ratio_df['roe'].map(lambda x: float(x.strip('%').replace(',',''))/100 if not pd.isnull(x) else np.nan)
charic_names=list(ratio_df.columns[2:])
ratio_df.dropna(inplace=True)

In [32]:
cluster_names=list(all_cluster.columns[1:])
all_df=all_cluster.merge(ratio_df,how='inner',on='company')

In [33]:
eval_df=pd.DataFrame(columns=['clustering']+charic_names)
total_comp=all_df.shape[0]
charic_total=[]
for r in range(len(charic_names)):
    charic_total.append(ratio_df['mkt_cap'].dot(ratio_df.iloc[:,r+2]))
charic_total=charic_total/ratio_df['mkt_cap'].sum()
cl_results=[]
for cl in range(len(cluster_names)):
    cl_res=[cluster_names[cl]]
    for r in range(len(charic_names)):
        clr_df=all_df.loc[:,[cluster_names[cl],charic_names[r],'mkt_cap']]
        clr_df['sumpro']=clr_df[charic_names[r]]*clr_df['mkt_cap']
        clr_df=clr_df.groupby(cluster_names[cl]).agg({charic_names[r]:'count','mkt_cap':'sum','sumpro':'sum'})
        clr_df['sigma']=clr_df[charic_names[r]]/np.sum(clr_df[charic_names[r]])*(clr_df['sumpro']/clr_df['mkt_cap']-charic_total[r])**2
        clr=np.sqrt(clr_df['sigma'].sum())
        cl_res.append(clr)
    cl_results.append(cl_res)
eval_df=eval_df.append(pd.DataFrame(cl_results,columns=eval_df.columns))
eval_df['average']=eval_df.mean(1)
eval_df.set_index('clustering',inplace=True)
eval_df=eval_df.sort_values('average', ascending=False)
print('Market value weighted:')
print(eval_df)

Market value weighted:
                 pb_ratio      beta  profit_m       roa       roe   average
clustering                                                                 
lsi_400_gmm     10.640692  0.338565  0.144305  0.052437  1.808450  2.596890
lsi_500_kmeans   9.826530  0.300953  0.128645  0.048725  2.046703  2.470311
lsi_200_gmm     10.232821  0.315177  0.146739  0.060368  1.020367  2.355094
lsi_300_kmeans   8.713562  0.313041  0.131408  0.052699  2.498582  2.341858
lsi_600_gmm      9.329555  0.288173  0.131433  0.053440  0.873638  2.135248
lsi_200_kmeans   7.531815  0.301557  0.128854  0.062301  1.811218  1.967149
lsi_500_gmm      6.879473  0.311960  0.141205  0.061411  1.814388  1.841687
lsi_600_kmeans   6.865131  0.321986  0.152246  0.059960  1.803690  1.840603
lsi_300_gmm      6.714773  0.311362  0.154679  0.044518  1.786671  1.802401
lsi_400_kmeans   6.307048  0.308758  0.130515  0.045149  2.171495  1.792593


In [34]:
eval_df=pd.DataFrame(columns=['clustering']+charic_names)
total_comp=all_df.shape[0]
charic_total=[]
for r in range(len(charic_names)):
    charic_total.append(ratio_df.iloc[:,r+2].mean())
cl_results=[]
for cl in range(len(cluster_names)):
    cl_res=[cluster_names[cl]]
    for r in range(len(charic_names)):
        clr_df=all_df.loc[:,[cluster_names[cl],charic_names[r]]]
        clr_df=clr_df.groupby(cluster_names[cl]).agg(['count','mean'])
        clr_df=clr_df[clr_df[(charic_names[r],'count')]>=5]
        clr_df['sigma']=clr_df[(charic_names[r],'count')]/np.sum(clr_df[(charic_names[r],'count')])*(clr_df[(charic_names[r],'mean')]-charic_total[r])**2
        clr=np.sqrt(clr_df['sigma'].sum())
        cl_res.append(clr)
    cl_results.append(cl_res)
eval_df=eval_df.append(pd.DataFrame(cl_results,columns=eval_df.columns))
eval_df['average']=eval_df.mean(1)
eval_df.set_index('clustering',inplace=True)
eval_df=eval_df.sort_values('average', ascending=False)
print('Equal value weighted:')
print(eval_df)

Equal value weighted:
                pb_ratio      beta  profit_m       roa       roe   average
clustering                                                                
lsi_500_kmeans  3.733019  0.411994  0.176020  0.073677  3.165244  1.511991
lsi_400_kmeans  4.306670  0.411335  0.295937  0.070271  2.399707  1.496784
lsi_600_kmeans  4.672141  0.407858  0.208312  0.074003  2.060186  1.484500
lsi_300_kmeans  4.003964  0.400623  0.188819  0.071592  2.112957  1.355591
lsi_300_gmm     3.665035  0.401511  0.264355  0.072710  2.351725  1.351067
lsi_600_gmm     4.125774  0.394775  0.246937  0.072379  1.756900  1.319353
lsi_500_gmm     3.674184  0.414296  0.314009  0.073091  2.068266  1.308769
lsi_200_kmeans  3.992022  0.380000  0.236780  0.073897  1.803764  1.297293
lsi_200_gmm     3.827782  0.401900  0.282234  0.074704  1.834449  1.284214
lsi_400_gmm     3.816172  0.412976  0.193261  0.073064  1.740735  1.247242


### Extra

In [ ]:
# Plotting for 2D
%pylab inline

xs = [w[0] for w in dtm_lsa]
ys = [w[1] for w in dtm_lsa]
xs, ys

figure()
plt.scatter(xs,ys)
xlabel('First principal component')
ylabel('Second principal component')
title('Plot of points against LSA principal components')

In [ ]:
def cosine_similarity(dtm):
    '''
    input: Doc Matrix
    output: list of lists indicating clusters
    '''
    corr = np.asarray(numpy.asmatrix(dtm_lsa) * numpy.asmatrix(dtm_lsa).T)
    
    L = dtm.shape[0]
    clusters = [[i] for i in range(L)]
    
    while (len(clusters) != NUM_INDUSTRIES):
        L = len(clusters)
        max_similarity = -1
        max_similarity_clusters = (None, None)
        for i in range(L):
            for j in range(i+1,L):
                cluster_1 = clusters[i]
                cluster_2 = clusters[j]
                n = len(cluster_1) * len(cluster_2)
                
                # Similarity
                total_similarity = 0
                for cpy_1 in cluster_1:
                    for cpy_2 in cluster_2:
                        print(cpy_1, cpy_2)
                        total_similarity += corr[cpy_1][cpy_2]
                similarity = total_similarity / n
                
                if similarity > max_similarity:
                    max_similarity = similarity 
                    max_similarity_clusters = (i, j)
        
        # Merge max_similarity clusters
        cluster_merge = clusters.pop(max_similarity_clusters[1])
        clusters[max_similarity_clusters[0]].append(cluster_merge)
        print(L)
    return clusters